# Rapport ARN_Lab03

@Authors: Parisod Nathan & Lestiboudois Maxime

In [15]:
# Import des bibliothèques nécessaires
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [16]:
# Chargement des données
df_train_1 = pd.read_csv('EEG_mouse_data_1.csv')
df_train_2 = pd.read_csv('EEG_mouse_data_2.csv')
df_test = pd.read_csv('EEG_mouse_data_test.csv')

# Concaténation des données d'entraînement
df_train = pd.concat([df_train_1, df_train_2], ignore_index=True)

In [17]:
# Sélection des 25 caractéristiques (ici les 25 premières colonnes correspondant à des amplitudes)
features = df_train.columns[:25]
X_train = df_train[features]
y_train = df_train['state']

X_test = df_test[features]

KeyError: "['state'] not in index"

In [5]:
# Normalisation des données
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Conversion en DataFrame pour une utilisation simplifiée ultérieure
X_train_scaled = pd.DataFrame(X_train_scaled, columns=features)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Vérification des données
print(X_train_scaled.head())
print(X_test_scaled.head())

ValueError: could not convert string to float: 'n'